<a href="https://colab.research.google.com/github/codemeharder21/MachineLearning/blob/main/DCUNetProject/Datasets/DatasetExtractionAndSplitCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'casia-20-image-tampering-detection-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1226991%2F2048184%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240220%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240220T173520Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2c4616267d2fbb948ba2555dbe8e2b02d473b72cac5e0ca6a9285a2ac483c69e4a71bf40ad940bc1ddb1796bf0053dae751e2d3d1aa476a3aa88bc1fee85f4ebc236285671adca518daef9951e629add9296fd685057429fc40d5310fbd14aae9315515a5ad88bcf908283500b0289f29ae1984f873ebd30328b4358c6b70bb1e3e867c386559833524259d72a0814b8ccbb0607a0ba551e3a094903a526b2e71c57394daaab8670e10fa263c812143902c7edaf1345bbea96a9671e4e0b1402616f2d2988fc45819ee3c9702f270024dd41bb46a2b5ffddbeded5667c2c879c4d6505ea7a569afedd6ed929a5d456b8056d6ca49facec7b90c0d868e1e9753d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 2745530246 bytes downloaded
Downloaded and uncompressed: casia-20-image-tampering-detection-dataset
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from PIL import Image,ImageChops,ImageEnhance
import cv2

In [3]:
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt

In [21]:
cd ../Validation

/kaggle/working/Casia-Image-Tampering-Split-Dataset/Validation


[Errno 2] No such file or directory: 'validGt'
/kaggle/working/Casia-Image-Tampering-Split-Dataset/Validation


In [23]:
mkdir GroundTruth

In [4]:
def load_image(path,mask=False):
    image = Image.open(path)
    image = image.resize((384, 256))

    if mask:
        image = image.convert('L')
    else:
        image = image.convert('RGB')

    image = np.array(image)
    return image

In [46]:
myDataLoc='/kaggle/working/Casia-Image-Tampering-Split-Dataset/'
trainLoc=os.path.join(myDataLoc,"Train")
testLoc=os.path.join(myDataLoc,"Test")
validLoc=os.path.join(myDataLoc,"Validation")
trainTp=os.path.join(trainLoc,"Tp")
testTp=os.path.join(testLoc,"Tp")
validTp=os.path.join(testLoc,"Tp")
trainGt=os.path.join(trainLoc,"GroundTruth")
testGt=os.path.join(testLoc,"GroundTruth")
validGt=os.path.join(validLoc,"GroundTruth")

In [47]:
xFolder='/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Tp'
yFolder='/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/CASIA 2 Groundtruth'
xTrain=[]
yTrain=[]
count =1
c=0
for fl in os.listdir(xFolder):
    try:
        imageID=os.path.basename(fl).split('.')[0]
        maskLoc=os.path.join(yFolder,f'{imageID}_gt.png')
        imageLoc=os.path.join(xFolder,fl)
        image=load_image(imageLoc)
        mask=load_image(maskLoc,True)
        # count+=1
        xTrain.append(image)
        yTrain.append(mask)
        # count+=1
        if count<3488:
            shutil.copy(imageLoc,trainTp)
            shutil.copy(maskLoc,trainGt)
        elif count<3986:
            shutil.copy(imageLoc,validTp)
            shutil.copy(maskLoc,validGt)
        else:
            shutil.copy(imageLoc,testTp)
            shutil.copy(maskLoc,testGt)
        count+=1
#         print("end")
    except:
        c+=1
        continue
print(count)
print(c)
xTrain=np.array(xTrain)
yTrain=np.array(yTrain)

4982
144


In [48]:
shutil.make_archive("Casia 2.0 Tampered Dataset", 'zip', myDataLoc)

'/kaggle/working/Casia 2.0 Tampered Dataset.zip'

In [34]:
import shutil

In [44]:
shutil.copy(os.path.join(xFolder,''), '/kaggle/working/MyF/')

'/kaggle/working/MyF/Tp_D_CND_S_N_txt00028_txt00006_10848.jpg'

In [43]:
mkdir MyF

mkdir GroundTruth

In [29]:
cd ..

/kaggle/working
